# Documentation

**Due: Thursday, Nov 8th at 11:59 PM**

## Introduction

Automatic differentiation (AD) is implemented in this software. More specifically, it can automatically differentiate of a python function up to machine precision and it can take derivatives of derivatives. Obtaining derivatives accurately is important because it is the key part of gradient-based optimization, which is the foundation of many machine learning algorithms. As a matter of fact, most machine learning problems can be divided into following steps:

1. define a function connecting some input $X$ with some output $Y$ with a set of parameters $\beta$ as $Y = f(X,\beta)$;
2. define a loss function to check how good the model is $L(X,Y,\beta)$;
3. find the parameter set $\beta$ that minimize the loss function: argmin$_\beta L(X,Y,\beta)$.

Generally the power or performance of a machine learning algorithm is limited by the third step which is can be handled by gradient-based optimization. Therefore, our software can be used in various machine learning packages and boost their performances.

Furthermore, AD can also be applied to solve differential equations in various physical systems. Such as, diffusion equations, wave equations, Navier–Stokes equations and other non-linear equations which cannot be solved analytically. Traditional numerical method using difference method possess error much larger than machine error. Therefore, applying AD will possibly increase the accuracy of the solvers of those differential equations. 

## Background

*Describe (briefly) the mathematical background and concepts as you see fit.  You **do not** need to
give a treatise on automatic differentiation or dual numbers.  Just give the essential ideas (e.g.
the chain rule, the graph structure of calculations, elementary functions, etc).*



#### What is AD?

AD is a set of techniques to numerically evaluate the derivative of a function specified by a computer program based on the fact that every computer program execute a sequence of elementary arithmetic operations and elementary functions. Using the chain rule, the derivative of each sub-expression can be calculated recursively to obtain the final derivatives. Depending on the sequence of calculating those sub-expressions, there are two major method of doing AD: **forward accumulation** and **reverse accumulation**. 

#### Why AD?

Traditionally, there are two ways of doing differentiation, i.e., symbolic differentiation (SD) and numerical differentiation (ND). SD gives exact expression of the derivatives and produce differentiation up to machine precision, while SD is very inefficient since the expression could become very during differentiation. ND on the other hand, suffers from round-off errors (or truncate error), which leads to bad precision. Moreover, both ND and SD have problems with calculating higher derivatives and they are slow for vector inputs with large size. AD solves all of these problems nicely.

#### How to do AD?

Considering a simple function:
$$z = \cos(x)\sin(y) + \frac{x}{y}$$
In AD, its computational graph for forward accumulation method looks like:
<img src="figs/Fig1.png" width="400">
Accoring to the graph, the simple function can be rewritten as

\begin{align}
z = \cos(x)\sin(y) + \frac{x}{y}=\cos(w_1) \sin(w_2) + \frac{w_1}{w_2}=w_3 w_4+w_6=w_5 + w_6=w_7
\end{align}
The derivates with respect to $x$ and $y$ can be calcualted according to chain rule as:

\begin{align}
\frac{\partial z}{\partial x}&=\frac{\partial z}{\partial w_7}\left(\frac{\partial w_7}{\partial w_5}\frac{\partial w_5}{\partial w_3}\frac{\partial w_3}{\partial w_1}+\frac{\partial w_7}{\partial w_6}\frac{\partial w_6}{\partial w_1}\right)\frac{\partial w_1}{\partial x}\\
\frac{\partial z}{\partial y}&=\frac{\partial z}{\partial w_7}\left(\frac{\partial w_7}{\partial w_5}\frac{\partial w_5}{\partial w_4}\frac{\partial w_4}{\partial w_2}+\frac{\partial w_7}{\partial w_6}\frac{\partial w_6}{\partial w_2}\right)\frac{\partial w_2}{\partial y}\\
\end{align}

Therefore $\frac{\partial z}{\partial x}$ and $\frac{\partial z}{\partial y}$ are just the combinations of derivatives of elementary functions, which can be calculated analytically. In forward accumulation, the chain rule are applied from inside to outside. Computationally, the values of $w_i$ and their derivatives are store along the chain accumulatively.

### Operator overloading

Operator overloading is applied in this package to realize AD. Values and derivatives of functions are updated and passed at each operation. That means if we have two functions $\omega_1 (x_1,x_2,...,x_n)$ and $\omega_2 (x_1,x_2,...,x_n)$ with their derivatives ${\partial\omega_1}/{\partial x_i}$ and ${\partial\omega_1}/{\partial x_i}$ and second derivatives ${\partial^2\omega_1}/{\partial x_i}{\partial x_j}$ and ${\partial^2\omega_2}/{\partial x_i}{\partial x_j}$, $\forall i,j$. For a new variable $\omega_3 = \omega_1*\omega_2$, where $*$ is some operator (addition, multiplication, division, power, etc). Accoring to specific types of the operator we can write down analytical expression of the values as well the derivatives of $\omega_3$ as functions of $\omega_1$ and $\omega_2$ and their derivatives. The following expression shows these analytical expressions:

| operator 	| &nbsp; &nbsp; &nbsp;value $\omega_3$&nbsp; | &nbsp; &nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; first derivative $\frac{\partial \omega_3}{\partial x_i}$  &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; &nbsp;| second derivative $\frac{\partial^2 \omega_3}{\partial x_i \partial x_j }$                                                                  	|   	|
|:---:|:---:|:---:|:---:|---	|
|    $+$   	|         $\omega_1 + \omega_2$         	|$\frac{\partial \omega_1}{\partial x_i} +\frac{\partial \omega_2}{\partial x_i}$                                    	|                                                                                                                                                                                                                                                                                     $\frac{\partial^2 \omega_1}{\partial x_i \partial x_j } +\frac{\partial^2 \omega_2}{\partial x_i \partial x_j }$                                                                                                                                                                                                                                                                                    	|   	|
|    $-$   	|    $\omega_1 - \omega_2$   	|                                    $\frac{\partial \omega_1}{\partial x_i} - \frac{\partial \omega_2}{\partial x_i}$                                   	|                                                                                                                                                                                                                                                                                     $ \frac{\partial^2 \omega_1}{\partial x_i \partial x_j } -\frac{\partial^2 \omega_2}{\partial x_i \partial x_j }$                                                                                                                                                                                                                                                                                    	|   	|
|    $\times$   	| $\omega_1 \times \omega_2$ 	|                           $ \omega_2 \frac{\partial \omega_1}{\partial x_i} +\omega_1\frac{\partial \omega_2}{\partial x_i}$                           	|                                                                                                                                                                                             $ \frac{\partial \omega_1}{\partial x_i}\frac{\partial \omega_2}{\partial x_j} + \frac{\partial \omega_1}{\partial x_j}\frac{\partial \omega_2}{\partial x_i}+ \omega_2 \frac{\partial^2 \omega_1}{\partial x_i \partial x_j } + \omega_1 \frac{\partial^2 \omega_2}{\partial x_i \partial x_j }$                                                                                                                                                                                            	|   	|
|    $/$   	|    $\omega_1 / \omega_2$   	|            $\frac{1}{\omega_2} \frac{\partial \omega_1}{\partial x_i} - \frac{\omega_1}{\omega_2^2} \frac{\partial \omega_2}{\partial x_i}$            	|                                                                                                        $ -\frac{1}{\omega_2^2}\left(\frac{\partial \omega_1}{\partial x_i}\frac{\partial \omega_2}{\partial x_j} + \frac{\partial \omega_1}{\partial x_j}\frac{\partial \omega_2}{\partial x_i}\right) + \frac{2\omega_1}{\omega_2^3}\frac{\partial \omega_2}{\partial x_i}\frac{\partial \omega_2}{\partial x_j}+\frac{1}{\omega_2} \frac{\partial^2 \omega_1}{\partial x_i \partial x_j } - \frac{\omega_1}{\omega_2^2} \frac{\partial^2 \omega_2}{\partial x_i \partial x_j }$                                                                                                        	|   	|
| ^     | $\omega_1^{\omega_2}$      	| $\omega_1 ^{\omega_2-1} \omega_2\frac{\partial \omega_1}{\partial x_i}+\ln(\omega_1) \omega_1^{\omega_2} \frac{\partial \omega_2}{\partial x_i}$ 	| $\omega_1^{\omega_2-2}(\omega_2^2 - \omega_2) \frac{\partial \omega_1}{\partial x_i}\frac{\partial \omega_1}{\partial x_j}+ (\omega_1^{\omega_2-1}+\omega_2\ln(\omega_1)\omega_1^{\omega_2-1})\left( \frac{\partial \omega_1}{\partial x_i}\frac{\partial \omega_2}{\partial x_j} +\frac{\partial \omega_1}{\partial x_j}\frac{\partial \omega_2}{\partial x_i}\right)+\ln (\omega_1)^2\omega_1^{\omega_2}\frac{\partial \omega_2}{\partial x_i}\frac{\partial \omega_2}{\partial x_j} + \omega_2 \omega_1 ^{\omega_2-1}\frac{\partial^2 \omega_1}{\partial x_i \partial x_j } + \ln(\omega_1) \omega_1^{\omega_2}\frac{\partial^2 \omega_2}{\partial x_i \partial x_j }$ 	|   	|



## How to Use *AutoDiff*

There are two ways to install our package.

### Method 1: User installation via ```pip```
Our package is currently published on PyPI, as ```AutoDiff-XVQD```. With the forward mode implementation complete, users can currently install our package via ```pip```. After creating a new directory for their project, the user should go into this directory and enter in the following commands:

Within the directory the user just created, they create a virtual environment and call it `env`.
```bash
virtualenv env
```

Then they activate the virtual environment and install the package.
```bash
source env/bin/activate
pip install AutoDiff-XVQD
```

After these steps, the user can open a Python interpreter on the virtual environment they just created and now they will be able to import the module.

```python
>>> import AutoDiff.AutoDiff as ad
```

### Method 2: Installation via Github (for developers and users)
Since our package is not on PyPI yet, the user needs to create a virtual environment and manually install the package. First, the user needs to download the package from GitHub. Then they create a directory and unpack the project into that directory. 
```bash
git clone https://github.com/XVQD/cs207-FinalProject.git
```
Within the directory the user just created, they create a virtual environment and call it `env`.
```bash
virtualenv env
```

Then they activate the virtual environment and install the dependencies.
```bash
source env/bin/activate
pip install -r requirements.txt
```

After these steps, the user can open a Python interpreter on the virtual environment they just created and now they will be able to import the module.

```python
>>> import AutoDiff.AutoDiff as ad
```

### Introduction to basic usage of the package

After successful installation, the user will first import our package.
```python
>>> import AutoDiff.AutoDiff as ad
```
Then depending on the type of expressions they have, they will employ one of the following methods.

#### Scalar functions of scalar values
Say the user wants to get the gradient of the expression $f(x) = alpha * x + 3$.
The user will first create a variable x and then define the symbolic expression for `f`.
```python
>>> x = ad.Variable(2, name='x')
>>> f = 2 * x + 3
```
Note: If the user wants to include special functions like sin and exp, they need to do the following:
```python
>>> f = 2 * ad.sin(x) + 3
```
Then when they want to evaluate the gradients of f with respect to x, they will do
```python
>>> print(f.val, f.der)
```
f.val and f.der will then contain the value and gradient of f with respect to x.

#### Scalar functions of vectors
Say the user wants to get the gradient of the expression $f(x1,x2) = x_1 x_2 + x_1$. 

The user will first create two variables `x1` and `x2` and then define the symbolic expression for `f`.
```python
>>> x1 = ad.Variable(2,name='x1')
>>> x2 = ad.Variable(3,name='x2')
>>> f = x1 * x2 + x_1
```
Then when they want to get the values and gradients of f with respect to x1 and x2, they will do
```python
>>> print(f.val, f.der)
```
f.val and f.der will then contain dictionaries of values and gradients of f with respect to x1 and x2.

#### Vector functions of vectors
Say the user wants to get the gradients of the system of functions 
$$f_1 = x_1 x_2 + x_1$$
$$f_2 = \frac{x_1}{x_2}$$

i.e.
$$\mathbf{f}(x1,x2)=(f_1(x_1,x_2),f_2(x_1,x_2))$$
The user will first create two variables `x1` and `x2` and then define the symbolic expression for `f`.
```python
>>> x1 = ad.Variable(3, name = 'x1')
>>> x2 = ad.Variable(2, name = 'x2')
>>> f1 = x1 * x2 + x_1
>>> f2 = x1 / x2
```
Then when they want to evaluate the gradients of f with respect to x1 and x2, they will do
```python
>>> print(f1.val, f2.val, f1.der, f2.der)
```
The Jacobian $\mathbf{J}(\mathbf{f})$ =(f1', f2') = (f1.der, f2.der)


### Demo

The following demo shows a simple case of using the AutoDiff package to calculate the derivatives. This can be done in the directory the user creates AFTER they install the package as shown in 'installation' section.

First the user imports the package `AutoDiff`
```python
>>> import AutoDiff.AutoDiff as ad
```

Then they define the variables. For example, if they have variables $x_1$ and $x_2$, which they want to evaluate at 3 and 2 respectively, they will define them as the following. The user needs to give a name as a parameter when they define the variables.
```python
>>> x1 = ad.Variable(3, name='x1')
>>> x2 = ad.Variable(2, name='x2')
```

Then the user writes down the expression/function they want to evaluate. Most of the expression would be normal arithmetic expressions, except for the special functions such as exp, sin, cos, etc., the user needs to use ad.exp(), ad.sin(), ad.cos(), etc.
```python
>>> f = ad.sin(x1) + x2**2
```

Finally, the user can get print the value and derivatives of their expression with respect to $x_1$ and $x_2$. This will output the value as a scalar and the partial derivatives as a dictionary where the keys are the variable names and the values are the derivatives with respect to that variable.
```python
>>> print(f.val, f.der)
4.141120008059867 {'x2': 4, 'x1': -0.9899924966004454}
```

## Software Organization

#### Directory structure 
```
/cs207-FinalProject
    /docs
        milestone1.ipynb
        milestone2.ipynb
    /AutoDiff
        __init__.py
        AutoDiff.py
    /tests
        __init__.py
        test_operator.py
        test_elementary_functions.py
    README.md
    requirements.txt
    LICENSE.md
```
#### Modules

- `__init__.py`:  initialize the package by importing necessary functions from other modules

- `AutoDiff.py`:  main module of the package which implements basic data structure and algorithms of the forward automatic differentiation, including overloaded operators and special functions such as sin and trig.

#### Test

The test suite will live on `test_operator.py` and `test_elementary_functions.py` files in tests folder. `test_operator.py` tests the overloaded operators and `test_elementary_functions.py` tests the elementary functions such as exp, cos, sin, etc. We automate our testing using continuous integration. Every time we commit and push to GitHub, our code is automatically tested by `Travis CI` and `Coveralls` for code coverage. 

#### Package Installation

Eventually we will use PyPI to distribute our package. At this point, the user needs to download and manually install the package as following.

First, the user needs to download the package from GitHub. Then they create a directory and unpack the project into that directory.
```bash
git clone https://github.com/XVQD/cs207-FinalProject.git
```

Within the directory the user just created, they create a virtual environment and call it env.
```bash
cd yourdir
virtualenv env
```

Then they activate the virtual environment and install the dependencies.
```bash
source env/bin/activate
pip install -r requirements.txt
```

After these steps, the user can open a Python interpreter on the virtual environment they just created and now they will be able to import the package.
```python
>>> import AutoDiff.AutoDiff as ad
```

## Implementation

### Current Implementation
#### Data structures
*What are the core data structures?*

* dictionary: we use dictionaries to keep track of the partial derivatives. The keys are the variables we differentiate with respect to and the values are the actual derivatives.
* overloaded operators such as \__add\__ and \__mul\__ to add or multiply two auto-differentiation objects.

#### Classes
*What are the core classes?*

* class Variable - an auto-differentiation class with the overloaded operators 
    * attributes
        * val: scalar value of current node
        * name: name of variable
        * der: dict of partial derivatives of current node
    * Methods
        * \__pos\__
        * \__neg\__
        * \__add\__
        * \__radd\__
        * \__sub\__
        * \__rsub\__
        * \__mul\__
        * \__rmul\__
        * \__itruediv\__
        * \__rtruediv\__
        * \__pow\__
        * \__rpow\__


* method exp()
    * input 
        * Variable object
    * output 
        * Variable object after taking exponential


* method log()
    * input 
        * Variable object
    * output 
        * Variable object after taking log


* method sin()
    * input 
        * Variable object
    * output 
        * Variable object after taking sine


* method cos()
    * input 
        * Variable
    * output 
        * Variable object after taking cosine


* method tan()
    * input 
        * Variable
    * output 
        * Variable object after taking tangent
        
* method sinh()
    * input 
        * Variable
    * output 
        * Variable object after taking sinh
      
      
* method cosh()
    * input 
        * Variable
    * output 
        * Variable object after taking cosh


* method tanh()
    * input 
        * Variable
    * output 
        * Variable object after taking tanh
        
        
* method arcsin()
    * input 
        * Variable
    * output 
        * Variable object after taking arcsin


* method arccos()
    * input 
        * Variable
    * output 
        * Variable object after taking arccos


* method arctan()
    * input 
        * Variable
    * output 
        * Variable object after taking arctan

#### External dependecies

* numpy==1.15.4
* scipy==1.1.0

#### Elementary functions

Our elementary functions include the following: 
* exp
* log
* sin
* cos
* tan
* sinh
* cosh
* tanh
* arcsin
* arccos
* arctan

## Future
*Now that you've got most of the hard implementation work done, what kinds of things do you want to impelement next? How will your software change? What will be the primary challenges?*

### Higher derivatives
At this point we stored the first derivative (gradient) of the function in a vector `f.der`. Principally we can also keep track of higher derivatives in vectors `f.der2`, `f.der3`, `f.der4`, etc. The specific implementation the higher derivatives will not change the exsiting code structure. The primary challenge would be define the overloaded operators and elementary functions for higher derivatives.

### Newton solver
In engineering as well as science, we always need to solve sets of nonlinear equations, i.e., find the solution of 

$$\mathbf{F}(\mathbf{X})=\mathbf{0}$$

where $\mathbf{X}$ and $\mathbf{F}$ are high dimensional vectors. Typically Newton's method is used to iteratively solve this equation,

$$\mathbf{X_{n+1}} = \mathbf{X_{n}} - \left(\mathbf{F'}(\mathbf{X_n})\right)^{-1} \mathbf{F}(\mathbf{X_n})$$

where $\mathbf{F'}(\mathbf{X_n}$ is the Jacobian of $\mathbf{F}$. The iteration ends when $\|\mathbf{F}(\mathbf{X_n})\|$ is sufficiently closed to zero.

The accuracy and robustness of Newton's method highly depend on the tolerence of the calculated Jocobian, which can be calculated using our AD package. As for implementation, we define function $\mathbf{F_0}$ with some initial guess of solution $\mathbf{X_0}$, then use Newton's method to find $\mathbf{X_1}$, then update the function to $\mathbf{F_1}$, then move on and on until $\mathbf{F_n}(\mathbf{X_n})$ is sufficiently closed to zero. 